### Concat all HIBP and Analyze

In [ ]:
import glob
import os
import pandas as pd

In [ ]:
# Directory path containing JSON files
directory = 'pwned/'

# Get list of JSON file paths in the directory
json_files = glob.glob(directory + '/*.json')

# List to store individual DataFrames
dataframes = []

# Iterate over the JSON files
for file in json_files:
    # Read JSON file into a DataFrame
    df = pd.read_json(file)
    # Append DataFrame to the list
    file_name = os.path.basename(file).rstrip('.json')
    df['file_name'] = file_name

    dataframes.append(df)

# Concatenate the list of DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
combined_df

In [ ]:
breaches = pd.read_json('breaches.json')

In [ ]:
fin_df = pd.merge(combined_df, breaches, on='Name')
fin_df.shape

In [ ]:
fin_df.groupby('file_name')['Name'].nunique().describe()

In [ ]:
fin_df[fin_df['IsSensitive']==True].groupby('file_name')['Name'].nunique().describe()

In [ ]:
fin_df